In [1]:
import os
import pandas as pd
import numpy as np

# 1. Load Data

In [2]:
os.chdir('../Data')
pd.set_option('display.max_columns', None)

df = pd.read_csv('events.csv')
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-09-24 11:57:06 UTC,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT
1,2020-09-24 11:57:26 UTC,view,139905,2144415926932472027,computers.components.cooler,zalman,17.16,1515915625519380411,tdicluNnRY
2,2020-09-24 11:57:27 UTC,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy
3,2020-09-24 11:57:33 UTC,view,635807,2144415923107266682,computers.peripherals.printer,pantum,113.81,1515915625519014356,aGFYrNgC08
4,2020-09-24 11:57:36 UTC,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ


# 2. Data Transformation

In [3]:
# split category_code into different category levels
df[['first_category', 'second_category', 'third_category']] = df['category_code'].str.split('.', 2, expand=True)
df[['category_code', 'first_category', 'second_category', 'third_category']].head()

,category_code,first_category,second_category,third_category
0,electronics.telephone,electronics,telephone,None
1,computers.components.cooler,computers,components,cooler
2,NaN,NaN,NaN,NaN
3,computers.peripherals.printer,computers,peripherals,printer
4,NaN,NaN,NaN,NaN


In [4]:
# drop rows with no first_category
df.dropna(subset=['first_category'], inplace=True)

# check distribution of event_type
df['event_type'].value_counts()

view        575506
cart         43626
purchase     29778
Name: event_type, dtype: int64

In [5]:
# create separate dataframes for each reward distribution
df1 = df.copy()
df2 = df.copy()
df3 = df.copy()

In [6]:
# assign rewards for each event_type
def assign_rewards_1(event):
    if (event=='view') or (event=='cart'):
        return 0
    else: # event=='purchase'
        return 1

def assign_rewards_2(event):
    if event=='view':
        return 0
    elif event=='cart':
        return 0.2
    else: # event=='purchase'
        return 1

def assign_rewards_3(event):
    if event=='view':
        return 0
    elif event=='cart':
        return 0.5
    else: # event=='purchase'
        return 1

df1['reward'] = df['event_type'].map(assign_rewards_1)
df2['reward'] = df['event_type'].map(assign_rewards_2)
df3['reward'] = df['event_type'].map(assign_rewards_3)

In [7]:
# map reward to each first_category
def map_rewards(df):
    for category in df['first_category'].unique():
        df[category] = np.where(df['first_category']==category, df['reward'], 0)
    return df

df1 = map_rewards(df1)
df2 = map_rewards(df2)
df3 = map_rewards(df3)

In [8]:
# keep first_category columns
df1 = df1.iloc[:, 13:]
df2 = df2.iloc[:, 13:]
df3 = df3.iloc[:, 13:]

In [9]:
df1.head()

,electronics,computers,construction,auto,appliances,stationery,furniture,kids,accessories,country_yard,apparel,medicine,sport,jewelry
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
df2.head()

,electronics,computers,construction,auto,appliances,stationery,furniture,kids,accessories,country_yard,apparel,medicine,sport,jewelry
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df3.head()

,electronics,computers,construction,auto,appliances,stationery,furniture,kids,accessories,country_yard,apparel,medicine,sport,jewelry
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# export cleaned data as csv file
df1.to_csv('events_cleaned_1.csv', index=False)
df2.to_csv('events_cleaned_2.csv', index=False)
df3.to_csv('events_cleaned_3.csv', index=False)